# GPT-2: Введение и претрейн

В этом ноутбуке мы изучим архитектуру GPT-2 (Generative Pre-trained Transformer 2) и научимся претрейнить небольшую языковую модель с нуля.

GPT-2 - это авторегрессионная языковая модель, которая была представлена OpenAI в 2019 году. Она основана на архитектуре трансформера и использует только декодерную часть (decoder-only), что делает её идеальной для задач генерации текста.

**План на сегодня: **
- Архитектура GPT-2 и принципы работы decoder-only трансформеров
- Использование предобученных моделей
- Подготовка данных для претрейнинга
- Полнуя реализация процесса обучения языковой модели
- Оценка качества обученной модели


In [ ]:
import os
import math
import random
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
import numpy as np
from transformers import (
    GPT2LMHeadModel,
    GPT2Tokenizer,
    GPT2Config,
    get_linear_schedule_with_warmup,
    get_cosine_schedule_with_warmup)
from datasets import load_dataset
from tqdm import tqdm
from pathlib import Path


current_dir = Path.cwd()


if current_dir.name == 'notebooks':
    project_dir = current_dir.parent  # E:\projects\spbu_dl_2025
    parent_dir = project_dir.parent   # E:\projects
else:
    project_dir = current_dir
    parent_dir = current_dir.parent

if str(parent_dir) not in sys.path:
    sys.path.insert(0, str(parent_dir))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

## 1. Архитектура GPT-2

GPT-2 использует архитектуру **decoder-only трансформера**. Это означает, что модель состоит только из декодерных слоев, в отличие от:
- **Encoder-only** (BERT): использует только энкодерные слои, подходит для задач понимания текста
- **Encoder-Decoder** (T5, BART): использует оба компонента, подходит для задач перевода и summarization

Вопросы:

- Почему GPT-2 использует causal masking, а не bidirectional attention как BERT?
- Какие преимущества дает decoder-only архитектура для генерации текста?
- В чем разница между self-attention в GPT-2 и cross-attention в encoder-decoder моделях?


## 2. Токенизация

GPT-2 использует **Byte Pair Encoding (BPE)** токенизацию. Это позволяет модели работать с подсловными единицами, что особенно полезно для редких слов и различных языков.

### Особенности GPT-2 токенизации:

- Использует BPE с претокенизацией (разбиение текста на чанки перед применением BPE)
- Словарь размером 50,257 токенов
- Специальные токены: `<|endoftext|>` для обозначения конца текста
- Не использует токены `[CLS]` и `[SEP]` как BERT


In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# GPT-2 не имеет pad_token по умолчанию, устанавливаем его
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("Специальные токены:")
print(f"pad_token: {tokenizer.pad_token}")
print(f"eos_token: {tokenizer.eos_token}")
print(f"bos_token: {tokenizer.bos_token}")
print(f"unk_token: {tokenizer.unk_token}")
print(f"\nРазмер словаря: {tokenizer.vocab_size}")


In [ ]:
# Примеры токенизации
texts = [
    "Hello, world!",
    "Привет, мир!",
    "I like to eat apples.",
    "Сколько букв в слове амальгамма?"
]

for text in texts:
    tokens = tokenizer.tokenize(text)
    token_ids = tokenizer.encode(text)
    decoded = tokenizer.decode(token_ids)
    
    print(f"\nТекст: {text}")
    print(f"Токены: {tokens}")
    print(f"ID токенов: {token_ids}")
    print(f"Декодированный: {decoded}")
    print("-" * 60)


In [ ]:
# Загружаем предобученную модель GPT-2 (small версия)
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.to(device)
model.eval()  # Переводим в режим оценки

print(f"Количество параметров: {sum(p.numel() for p in model.parameters()):,}")
print(f"Размер модели: {sum(p.numel() * 4 for p in model.parameters()) / 1024 / 1024:.2f} MB")


Размеры моделей GPT-2:

| Модель | Слои | Головы внимания | Размерность эмбеддингов | Параметры |
|--------|------|-----------------|------------------------|-----------|
| **GPT-2** | 12 | 12 | 768 | 124M |
| **GPT-2 Medium** | 24 | 16 | 1024 | 355M |
| **GPT-2 Large** | 36 | 20 | 1280 | 774M |
| **GPT-2 XL** | 48 | 25 | 1600 | 1.5B |

Параметры конкретно нашей модельки:

In [ ]:
# Просмотр конфигурации GPT-2 124B
config = model.config
print("Конфигурация GPT-2:")
print(f"vocab_size: {config.vocab_size}")
print(f"n_positions: {config.n_positions} (максимальная длина последовательности)")
print(f"n_ctx: {config.n_ctx} (контекстное окно)")
print(f"n_embd: {config.n_embd} (размерность эмбеддингов)")
print(f"n_layer: {config.n_layer} (количество трансформерных слоев)")
print(f"n_head: {config.n_head} (количество голов внимания)")
print(f"n_inner: {config.n_inner} (размерность внутреннего слоя FFN)")
print(f"activation_function: {config.activation_function}")
print(f"layer_norm_epsilon: {config.layer_norm_epsilon}")


In [ ]:
print(model)

In [ ]:
state_dict = model.state_dict()

In [ ]:
# Простая генерация текста
def generate_text(prompt, max_length=50, temperature=1.0, top_k=50, top_p=0.95, do_sample=True):
    inputs = tokenizer.encode(prompt, return_tensors='pt').to(device)
    
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=max_length,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=do_sample
        )
    
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

# Пример генерации
prompt = "Cats are very cute animals."
print("Сгенерированный текст:")
print(generate_text(prompt, max_length=80))


Параметры генерации:

- **temperature**: контролирует случайность генерации (выше = более случайно, ниже = более детерминированно)
- **top_k**: ограничивает выбор следующих токенов только k наиболее вероятными
- **top_p (nucleus sampling)**: выбирает токены из минимального набора, сумма вероятностей которых >= p

Попробуем изменить параметры генерации и посмотрим, как это влияет на результат:


In [ ]:
# Экспериментируйте с параметрами
prompt = "Once upon a time"

print("Temperature = 0.5 (более детерминированно):")
print(generate_text(prompt, max_length=60, temperature=0.5))
print("\n" + "="*60 + "\n")

print("Temperature = 1.5 (более случайно):")
print(generate_text(prompt, max_length=60, temperature=1.5))
print("\n" + "="*60 + "\n")

print("Top-k = 10 (более ограниченный выбор):")
print(generate_text(prompt, max_length=60, top_k=10))
print("\n" + "="*60 + "\n")

print("Top-p = 0.5 (nucleus sampling):")
print(generate_text(prompt, max_length=60, top_p=0.5))


Вопросы:

- Как увеличение количества слоев влияет на способности модели?
- Почему размерность эмбеддингов обычно кратна количеству голов внимания?
- Какие компромиссы существуют между размером модели и скоростью обучения/инференса?

## GPT-2 с нуля

Для начала рассмотрим исходную архитектуру трансформера:

<img src='../images/transformer.png' width='500px' />

Начнем реализовывать нашу модельку, для начала повторим структуру модели из huggingface, чтобы 1) научиться генерировать 2) удостовериться, что наша модель будет работать как ожидается, когда мы ее обучим. 3) Не обучать ее полностью с нуля далее. 

Отличия модели от исходной архитектуры Transformer:
1) Только decoder-часть
2) Используется pre-normalization, не post-normalization. Это помогает в обучении, так как стабилизирует градиенты за счет более сбалансированного вклада residual ветки и основной в граф. Кроме того, в таком случае на вход MLP и Attention приходит отнормализованный сигнал, что также улучшает стабильность.
3) В качестве активации используется [GeLU](https://arxiv.org/pdf/1606.08415), так как она решает проблему мертвых градиентов в ReLU, дифференцируема везде и немного помогает отрегуляризировать модель
Формула GeLU: $f(x) = x · \Phi(x) $, где $\Phi(x)$ - CDF распределения Гаусса

Особенности:
- для полного воспроизведения результатов используется старая верся GeLU, которая основана на аппроксимации с помощью [tanh](https://docs.pytorch.org/docs/stable/generated/torch.nn.GELU.html). 
- Можете изучить исходный код [huggingface](https://github.com/huggingface/transformers/blob/main/src/transformers/models/gpt2/modeling_gpt2.py), который во многом повторяется здесь.

In [ ]:
from dataclasses import dataclass
@dataclass
class GPTConfig:
    vocab_size: int = 50257 # почему именно 50257?
    block_size: int = 1024 # corresponding to max sequence length
    n_layer: int = 12 # number of layers
    n_head: int = 12 # number of attention heads
    n_embed: int = 768 # embedding dimension
    activation_function: str = "gelu" # activation function
    layer_norm_epsilon: float = 1e-5 # epsilon for layer normalization
    use_torch_attention: bool = False
    n_positions: int = 1024 # maximum sequence length


# Если вы решите учить модель с нуля, можете сравнить разные функции активации
activations = {
    "gelu": nn.GELU(),
    "relu": nn.ReLU(),
    "silu": nn.SiLU(),
    "swi": nn.SiLU(),
    "gelu_old": nn.GELU(approximate="tanh"),
}



Для начала создадим скелет модели. GPT-2 представляет собой достаточно типичную структуру, в которой последовательно повторяется некоторое количество одинаковых трансформерных блоков. Названия слоев и блоков, а также все параметры должны быть такими же, как у предобученной модели.

In [ ]:
from spbu_dl_2025.utils.llm import sample_next_token

class GPT(nn.Module):
    def __init__(self, config: GPTConfig):
        super().__init__()
        self.config = config
        self.use_torch_attention = config.use_torch_attention
        # wpe means positional encoding, wte means token embedding (output embedding), h means hidden layers, ln_f means final layer norm
        self.transformer = nn.ModuleDict({
            'wpe': # создайте слой эмбеддинга для кодирования позиций,
            'wte': #  создайте слой эмбеддингов для кодирования токенов,
            'h': # создайте внутреннюю часть (последовательность слоев TransformerBlock) с помощью nn.ModuleList
            'ln_f': nn.LayerNorm(config.n_embed, eps=config.layer_norm_epsilon),
        })
        self.lm_head = #Cоздайте голову для определения логитов токеноы

        # weight sharing scheme (reduces 768*50267=~40M params, fewer params, more efficient)
        self.transformer.wte.weight = self.lm_head.weight
        
        for block in self.blocks:
            x = block(x)
        x = self.transformer.ln_f(x)

        _, seq_len = idx.size()
        assert seq_len <= self.config.block_size, f"Cannot forward sequence of length {T}, block size is only {self.config.block_size}"

        pos = # Создайте входы для позиционных эмбеддингов. Позиционные эмбеддинги принимают на вход номера токенов в последовательности
        pos_emd = self.transformer.wpe(pos)
        tok_emd = self.transformer.wte(idx)

        x = # Сложите эмбеддинги
        for block in self.transformer.h:
            x = block(x)
        x = self.transformer.ln_f(x)

        # For model train
        loss = None
        if targets is not None:
            # Зачем необходимо использование view в этой части?
            loss = F.cross_entropy(self.lm_head(x).view(-1, self.config.vocab_size), targets.view(-1))
            
        return self.lm_head(x), loss

    def generate(
        self, 
        x, 
        max_length, 
        do_sample=True, 
        top_k=None, 
        top_p=None, 
        temperature=1.0,
        pad_token_id=None,
        eos_token_id=None,
        ):

        while x.size(1) < max_length:
            if x.size(1) >= self.config.block_size:
                break
            logits, _ = self.forward(x)
            logits = # возьмите логиты только последнего токена
            new_token = sample_next_token(logits, do_sample, top_k, top_p, temperature)
                
            x = torch.cat([x, new_token], dim=1)
            # Check if any token in the batch is EOS token
            if eos_token_id is not None and (new_token == eos_token_id).any():
                # EOS stopping criterion
                break
            # Опционально: реализуйте генерацию в батче
        return x


    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            std = 0.02
            if hasattr(module, 'SCALE_INIT'):
                std /= (2 * self.config.num_layers)**0.5
            torch.nn.init.normal_(module.weight, mean=0, std=std)    # as per openai gpt-2 source code
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0, std=0.02)
    
    
    @classmethod
    def from_pretrained(cls, model_type, use_torch_attention: bool = False, use_grad_checkpoint: bool = False):
        """Loads pretrained weights from Hugging Face."""
        assert model_type in {"gpt2", "gpt2-medium", "gpt2-large", "gpt2-xl"}
        print("loading weights from pretrained gpt: %s" % model_type)

        # n_layer, n_head and n_embd are determined from model_type
        config_args = {
            "gpt2": dict(n_layer=12, n_head=12, n_embed=768),  # 124M params
            "gpt2-medium": dict(n_layer=24, n_head=16, n_embed=1024),  # 350M params
            "gpt2-large": dict(n_layer=36, n_head=20, n_embed=1280),  # 774M params
            "gpt2-xl": dict(n_layer=48, n_head=25, n_embed=1600),  # 1558M params
        }[model_type]
        config_args["vocab_size"] = 50257  # always 50257 for GPT model checkpoints
        config_args["block_size"] = 1024  # always 1024 for GPT model checkpoints
        config_args["use_torch_attention"] = use_torch_attention

        config = GPTConfig(**config_args)
        model = GPT(config)
        sd = model.state_dict()
        sd_keys = sd.keys()
        sd_keys = [k for k in sd_keys if not k.endswith(".attn.bias")]  # discard this mask / buffer, not a param

        # init a huggingface/transformers model
        model_hf = GPT2LMHeadModel.from_pretrained(model_type)
        sd_hf = model_hf.state_dict()
        # copy while ensuring all of the parameters are aligned and match in names and shapes
        sd_keys_hf = sd_hf.keys()
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith(".attn.masked_bias")]  # ignore these, just a buffer
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith(".attn.bias")]  # same, just the mask (buffer)
        transposed = ["attn.c_attn.weight", "attn.c_proj.weight", "mlp.c_fc.weight", "mlp.c_proj.weight"]
        # basically the openai checkpoints use a "Conv1D" module, but we only want to use a vanilla Linear
        # this means that we have to transpose these weights when we import them
        assert len(sd_keys_hf) == len(sd_keys), f"mismatched keys: {len(sd_keys_hf)} != {len(sd_keys)}"
        for k in sd_keys_hf:
            if any(k.endswith(w) for w in transposed):
                # special treatment for the Conv1D weights we need to transpose
                assert sd_hf[k].shape[::-1] == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k].t())
            else:
                # vanilla copy over the other parameters
                assert sd_hf[k].shape == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k])

        return model


Реализуем сам TransformerBlock и MLP:
Устройство TransformerBlock аналогично базовому блоку (Однако, по крайней мере в transformers, используется нормализация до подачи в аттеншен, а не после)

In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self, config: GPTConfig):
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.n_embed, eps=config.layer_norm_epsilon)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = nn.LayerNorm(config.n_embed, eps=config.layer_norm_epsilon)
        self.mlp = MLP(config)
    
    def forward(self, x):
        # normalization is applied before the attention and mlp
        # this helps to stabilize the training because the gradients are less likely to explode
        # it also helps to improve the performance of the model because inputs 
        # of attention and mlp are normalized

        x = # Реализуйте аттеншен часть с пренормализацией
        x = # Реализуйте линейную часть с пренормализацией
        return x


class MLP(nn.Module):
    def __init__(self, config: GPTConfig):
        super().__init__()
        self.c_fc = # Линейный слой с 4х-кратным расширением
        self.c_proj =# Линейный слой с 4х-кратным сужением
        # self.act = activations[config.activation_function] - на будущее
        self.gelu = nn.GELU(approximate="tanh")

    def forward(self, x):
        return self.c_proj(self.gelu(self.c_fc(x))) # для загрузки весов



Последняя фундаментальная часть - слой Attention, в нашем случае Multihead Self Attention. 

In [ ]:

class CausalSelfAttention(nn.Module):
    def __init__(self, config: GPTConfig):
        super().__init__()
        assert config.n_embed % config.n_head == 0, "n_embed must be divisible by n_head to make the heads equal size"
        self.n_head = config.n_head
        self.n_embed = config.n_embed
        self.c_attn = nn.Linear(config.n_embed, 3 * config.n_embed) # 3 * n_embed because we have 3 submatrices: query, key, value
        self.c_proj = nn.Linear(config.n_embed, config.n_embed) 
        self.use_torch_attention = config.use_torch_attention
        # In openai it was "bias" so i need to use that name, but actually it's a mask
        self.register_buffer('bias', 
        torch.triu(
            # создайте единичную марицу типа bool размера block_size x block size, замените верхний трегольник на нули.
        ).view(1, 1, config.block_size, config.block_size)
    ) # 4 dimensions for mask are: (batch, head, tokens, tokens), and one mask is used for all heads and all sequence

    def forward(self, x):
        batch, tokens, emb_dim = x.size() # batch size, sequence length, embedding dimensionality (n_embed)    

        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        # question: why we need to use one c_attn to calculate q,k,v? 
        q, k, v = self.c_attn(x).split(self.n_embed, dim=2)
        # question: why we need to split the input by n_embed?
        k = k.view(batch, tokens, self.n_head, emb_dim // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        q = q.view(batch, tokens, self.n_head, emb_dim // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(batch, tokens, self.n_head, emb_dim // self.n_head).transpose(1, 2) # (B, nh, T, hs)    

        # causal self-attention; Self-attend: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
        if self.use_torch_attention:  
            # efficient attention using Flash Attention if it's available
            out = F.scaled_dot_product_attention(q, k, v, attn_mask=None)
        else:
            # question: why we need to scale the attention?
            att = # Посчитайте (q * k^T) / sqrt(d)
            att = # примените каузальную маску, заполните скоры "-inf"
            att = # примените софтмакс
            out = # Посчитайте финальные скоры, умножив out на v # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
        
        out = out.transpose(1, 2).contiguous().view(batch, tokens, self.n_embed) # we "concatenate" all heads outputs side by side

        # output projection
        return self.c_proj(out) 


Проверим нашу модельку:

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
model = GPT.from_pretrained("gpt2")
model.eval()
model.to(device)


In [ ]:
print("Top-k = 20:")
prompt = "Where is Saint Petersburg?"
print(generate_text(prompt, max_length=60, top_k=20))
print("\n" + "="*60 + "\n")
print(generate_text(prompt, max_length=60, top_k=20))

print("\n" + "="*60 + "\n")
print(generate_text(prompt, max_length=60, top_k=20))


Итак, мы смогли создать модель, которая может генерировать текст, теперь нам нужно научиться ее обучать.

In [ ]:
print("Top-k = 20:")
prompt = "Где находится Москва?"
print(generate_text(prompt, max_length=60, top_k=20))
print("\n" + "="*60 + "\n")
print(generate_text(prompt, max_length=60, top_k=20))

print("\n" + "="*60 + "\n")
print(generate_text(prompt, max_length=60, top_k=20))

## 5. Претрейн
В этом ноутбуке мы не будем на самом деле претрейнить модель, но мы попробуем просимулировать это на небольшом корпусе русского языка, инициализировав модель >!квеном!<.
Я взяла [artifitial dostojevsky](https://gitlab.com/z00logist/artificial-dostoevsky/-/blob/main/data/corpus.txt?ref_type=heads), но можно использовать и, например, другие корпусы:
- [стихотворения Пушкина](https://dataverse.pushdom.ru/dataset.xhtml?persistentId=doi:10.31860/openlit-2023.8-C005)
- [финансовые новости](https://www.kaggle.com/datasets/kkhubiev/russian-financial-news)
- [19 000 Russian poems](https://www.kaggle.com/datasets/grafstor/19-000-russian-poems)
- [Russian Novels](https://github.com/JoannaBy/RussianNovels/tree/master) - классические произведения (около 100)

и другие...

In [ ]:

with open(r"E:\projects\spbu_dl_2025\notebooks\data\corpus.txt", "r", encoding="utf-8") as f:
    corpus = f.read()
corpus[:100]

Как будем учить? У нас есть единый корпус, но нам надо:
1) разбить его на входы и лейблы
3) собрать в батчи
4) настроить оптимизатор, шедулер и лосс
5) собственно, подождать N часов/дней, пока модель обучится

Как выглядят входы (x) и лейблы (y) модели:
x – фрагмент последовательности (токенов), начинающийся с любого индекса всей обучающей последовательности и размером seq_len.
y – это тот же x, только смещенный на одну позицию вправо:

In [ ]:
data = tokenizer.encode(corpus)

In [ ]:
len(corpus)/1000000, len(data)/1000000, "M"

Проверим обучение на 1 батче:

In [ ]:
batch_size = 4
seq_len = 1024
num_training_steps = 50
learning_rate = 5e-4

In [ ]:
x = torch.tensor(data[0: seq_len * batch_size]).view(batch_size, seq_len).to(device)
y = torch.tensor(data[1: seq_len * batch_size+1]).view(batch_size, seq_len).to(device)
x.shape, y.shape

Для оценки моделей часто используется perplexity. Эта функция определяется как экспонента от среднего NLL последовательности (т.е. нашей функции потерь). Интуитивный смысл этой функции в том, что она показывает, насколько уверенно модель предсказывает следующий токен.

In [ ]:
def calculate_perplexity(loss):
    """Вычисление perplexity из loss"""
    return math.exp(loss)


Далее начинается уже знакомый нам пайплайн обучения, так как задача фактически свелась к бинарной классификации. 

In [ ]:
optimizer = AdamW(model.parameters(), lr=learning_rate)

# Выберите тип scheduler:
# Вариант 1: Linear schedule
# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=100, num_training_steps=num_training_steps)

# Вариант 2: Cosine schedule (рекомендуется для длительного обучения)
scheduler = get_cosine_schedule_with_warmup(
    optimizer, 
    num_warmup_steps=100, 
    num_training_steps=num_training_steps,
    num_cycles=0.5  # Половина цикла косинуса
)

for i in range(num_training_steps):
    optimizer.zero_grad()
    logits, loss = model(x, y)
    loss.backward()
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
    print(f"Step {i} | loss: {loss.item()}, perplexity: {calculate_perplexity(loss.item())}")

In [ ]:
prompt = "Ох уж мне эти сказочники!"
print("\n" + "="*60 + "\n")
print(generate_text(prompt, max_length=128, top_k=20))


Обучение мы будем проводить с помощью Pytorch, как мы это уже делали раньше. Поэтому нам снова нужно подготовить класс датасета, который будет отдавать один семпл данных. 
Для нас разумно разбивать данные с перекрытием, а именно считать индекс семпла как индекс его первого токена, и отдавать семпл по этому индексу. Тогда, если мы не зайдем за край датасета, мы сможем наиболее эффективно его использовать.


In [ ]:

class PretrainData(Dataset):
    """
    Создает пары (x, y) из последовательности данных, где:
    - x: последовательность длиной seq_len, начиная с позиции idx
    - y: последовательность длиной seq_len, начиная с позиции idx + 1 (сдвиг на 1)
    """
    def __init__(self, data: list[int], seq_len: int, device: str):
        self.data = data
        self.seq_len = seq_len
        self.device = device
    
    def __len__(self):
        return len(self.data) - self.seq_len - 1 # нельзя заходить за край датасета
    
    def __getitem__(self, idx: int):
        x = # создайте long tensor из данных размера seq_len, начиная с idx
        y = # создайте long tensor из данных размера seq_len, начиная с idx + 1
        return (x, y)

Вы можете попробовать обучить модель совсем с нуля, для этого вам понадобятся гораздо более крупные датасеты (и гораздо больше ресурсов). Вот пара примеров:

In [ ]:
dataset = PretrainData(data, seq_len=1024, device=device)
# Разделяем на train и validation
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(
    dataset, [train_size, val_size]
)
batch_size=4
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


print(f"Train размер: {len(train_dataset)}")
print(f"Validation размер: {len(val_dataset)}")

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

print(f"Train число батчей: {len(train_loader)}")
print(f"Validation число батчей: {len(val_loader)}")



Для обучения языковых моделей часто используются два типа learning rate schedulers:

1. **Linear Schedule** (`get_linear_schedule_with_warmup`):
   - Линейно уменьшает learning rate от максимального значения до 0
   - Простая и стабильная стратегия
   - Хорошо работает для короткого обучения

2. **Cosine Schedule** (`get_cosine_schedule_with_warmup`):
   - Уменьшает learning rate по косинусоиде
   - Более плавное уменьшение в начале, более быстрое в конце
   - Часто дает лучшие результаты для длительного обучения
   - Параметр `num_cycles` контролирует количество циклов косинуса:
     - `num_cycles=0.5` (по умолчанию) - половина цикла (от 0 до π)
     - `num_cycles=1.0` - полный цикл (от 0 до 2π)
     - `num_cycles=1.5` - полтора цикла

**Warmup** - это период, в течение которого learning rate постепенно увеличивается от 0 до максимального значения. Это помогает стабилизировать обучение в начале. (У нас обучение идет не с начала, но мы все равно настроим его)


В этом ноутбуке предполагается некотое количество экспериментов и абляций, поэтому настраиваем ClearML (альтернатива wandb)

In [ ]:
%env CLEARML_WEB_HOST=https://app.clear.ml/
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY=#your_key
%env CLEARML_API_SECRET_KEY=#your_key

In [ ]:
from clearml import Task
task = Task.init(project_name="gpt2-pretrain", task_name="Experiment Run 2")

Зададим параметры обучения и модели:

In [ ]:
params={
    "batch_size": batch_size,
    "num_epochs": 2,
    "seq_len": seq_len,
    "learning_rate": learning_rate,
    "num_training_steps": num_training_steps,
    "num_warmup_steps": 100,
    "optimizer": "AdamW",
    "scheduler": "cosine_with_warmup",
    "gradient_accumulation": 4,
    "learning_rate": 5e-4,
    "num_warmup_steps": 1000,
    "use_torch_attention": True,
    "use_grad_checkpoint": True

}

task.connect(params)

In [ ]:
model = GPT.from_pretrained("gpt2", use_torch_attention=params["use_torch_attention"], use_grad_checkpoint=params["use_grad_checkpoint"])
model.eval()
model.to(device)


In [ ]:
# Настройка обучения
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Используемое устройство: {device}")

model = model.to(device)

# Оптимизатор
learning_rate = params["learning_rate"]
optimizer = AdamW(model.parameters(), lr=learning_rate)

# Learning rate scheduler
num_epochs = params["num_epochs"]
num_training_steps = len(train_loader) * num_epochs
num_warmup_steps = params["num_warmup_steps"]

if params["scheduler"] == "linear_with_warmup":
    # Linear schedule (линейное уменьшение learning rate)
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=num_warmup_steps,
        num_training_steps=num_training_steps
    )
elif params["scheduler"] == "cosine_with_warmup":
    # Cosine schedule уменьшает learning rate по косинусоиде, что часто дает лучшие результаты
    # Особенно полезно для длительного обучения
    scheduler = get_cosine_schedule_with_warmup(
        optimizer,
        num_warmup_steps=num_warmup_steps,
        num_training_steps=num_training_steps,
        num_cycles=0.5  # Количество полных циклов косинуса (0.5 = половина цикла)
    )
else:
    raise ValueError("scheduler is not configured")


In [ ]:
# Функция для валидации
def validate(model, val_loader, device,  max_batches: int = 100):
    model.eval()
    total_loss = 0
    total_tokens = 0
    
    with torch.no_grad():
        for batch_id, batch in enumerate(val_loader):
            if batch_id > max_batches:
                break
            inputs, targets = batch

            preds, loss = model(inputs, targets=targets)
            
            # Учитываем только не-padding токены
            mask = (targets != tokenizer.pad_token_id).float()
            num_tokens = mask.sum().item()
            
            total_loss += loss.item() * num_tokens
            total_tokens += num_tokens
    
    avg_loss = total_loss / total_tokens if total_tokens > 0 else 0
    perplexity = calculate_perplexity(avg_loss)
    
    return avg_loss, perplexity


Собираем все вместе:

In [1]:
def run_exp(
    exp_name, 
    task,
    batch_size: int = 4, 
    gradient_accumulation: int = 4, 
    num_warmup_steps: int = 100, 
    scheduler: str = "cosine_with_warmup",
    use_torch_attention: bool = True,
    use_grad_checkpoint: bool = False,
    max_batches: int = None,
    save_model: bool = True
    ) -> None:
    
    if Task.current_task() is not None:
        Task.current_task().close()
    task = Task.init(project_name="gpt2-pretrain", task_name=f"Experiment Run {exp_name}")

    params={
        "batch_size": batch_size,
        "num_epochs": 1,
        "seq_len": seq_len,
        "optimizer": "AdamW",
        "scheduler": scheduler,
        "gradient_accumulation": gradient_accumulation,
        "learning_rate": 5e-4,
        "num_warmup_steps": num_warmup_steps,
        "use_torch_attention": use_torch_attention,
        "use_grad_checkpoint": use_grad_checkpoint

    }

    task.connect(params)

    model = GPT.from_pretrained("gpt2", use_torch_attention=params["use_torch_attention"])
    model.eval()
    model.to(device)

    batch_size = params["batch_size"]
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


    learning_rate = params["learning_rate"]
    optimizer = AdamW(model.parameters(), lr=learning_rate)

    num_epochs = params["num_epochs"]
    num_training_steps = len(train_loader) * num_epochs
    num_warmup_steps = params["num_warmup_steps"]

    if params["scheduler"] == "linear_with_warmup":
        # Вариант 1: Linear schedule (линейное уменьшение learning rate)
        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=num_warmup_steps,
            num_training_steps=num_training_steps
        )
    elif params["scheduler"] == "cosine_with_warmup":
        # Вариант 2: Cosine schedule (косинусное изменение learning rate)
        # Cosine schedule уменьшает learning rate по косинусоиде, что часто дает лучшие результаты
        # Особенно полезно для длительного обучения
        scheduler = get_cosine_schedule_with_warmup(
            optimizer,
            num_warmup_steps=num_warmup_steps,
            num_training_steps=num_training_steps,
            num_cycles=0.5  # Количество полных циклов косинуса (0.5 = половина цикла)
        )
    else:
        raise ValueError("scheduler is not configured")

    print("Начинаем обучение...\n")

    train_losses = []
    val_losses = []
    val_perplexities = []
    gradient_accumulation_steps = params["gradient_accumulation"]

    for epoch in range(params["num_epochs"]):
        model.train()
        epoch_loss = 0
        num_batches = 0
        
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
        
        for batch_idx, batch in enumerate(progress_bar):
            if max_batches is not None and batch_idx > max_batches:
                break
            inputs, targets = batch
            preds, loss = model(inputs, targets=targets)

            loss.backward()

            if (batch_idx + 1) % gradient_accumulation_steps == 0:
                # Зачем необходимо клипать градиенты?
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                # Сделайте шаги оптимизатора и шедулера
            
            epoch_loss += loss.item()
            num_batches += 1
            
            current_lr = scheduler.get_last_lr()[0]
            progress_bar.set_postfix({'loss': loss.item()})
            
            # Log batch-level metrics to ClearML
            if 'task' in globals():
                global_step = epoch * len(train_loader) + batch_idx
                task.logger.report_scalar(
                    title="Training/Batch",
                    series="Loss",
                    value=loss.item(),
                    iteration=global_step
                )
                task.logger.report_scalar(
                    title="Training/Batch",
                    series="Learning Rate",
                    value=current_lr,
                    iteration=global_step
                )
        
        avg_train_loss = epoch_loss / num_batches
        train_losses.append(avg_train_loss)
        
        # Валидация
        val_loss, val_ppl = validate(model, val_loader, device)
        val_losses.append(val_loss)
        val_perplexities.append(val_ppl)
        
        print(f"\nEpoch {epoch+1}/{num_epochs}:")
        print(f"  Train Loss: {avg_train_loss:.4f}")
        print(f"  Val Loss: {val_loss:.4f}")
        print(f"  Val Perplexity: {val_ppl:.2f}\n")
        
        # Log epoch-level metrics to ClearML
        if 'task' in globals():
            task.logger.report_scalar(
                title="Training/Epoch",
                series="Loss",
                value=avg_train_loss,
                iteration=epoch + 1
            )
            task.logger.report_scalar(
                title="Validation/Epoch",
                series="Loss",
                value=val_loss,
                iteration=epoch + 1
            )
            task.logger.report_scalar(
                title="Validation/Epoch",
                series="Perplexity",
                value=val_ppl,
                iteration=epoch + 1
            )
            task.logger.report_scalar(
                title="Training/Epoch",
                series="Learning Rate",
                value=scheduler.get_last_lr()[0],
                iteration=epoch + 1
            )

    print("Обучение завершено!")

    # Тестируем генерацию
    test_prompts = [
        "The future of",
        "Machine learning",
        "Ох уж эти мне сказочники! ",
        "Петербург выглядел ",
        "Где находится Москва? "
    ]

    print("Генерация текста обученной моделью:\n")
    for prompt in test_prompts:
        generated = generate_text(prompt, max_length=60, top_k=20)
        print(f"Prompt: '{prompt}'")
        print(f"Generated: {generated}\n")
        print("-" * 60)


    if save_model:
        save_path = "./gpt2_pretrained"
        torch.save(model.state_dict(), save_path+f"/model_{exp_name}.pth")
        print(f"Модель сохранена в {save_path} с именем model_{exp_name}.pth")

In [ ]:
seq_len = 1024
run_exp(0, task, use_grad_checkpoint=False)

In [ ]:
i = 8
for gradient_accumulation in [4, 8, 16]:
    for num_warmup_steps in [10, 20]:
        for scheduler in ["cosine_with_warmup", "linear_with_warmup"]:
            run_exp(i, task, gradient_accumulation=gradient_accumulation, num_warmup_steps=num_warmup_steps, scheduler=scheduler)
            i += 1


Вопросы:
.
- Почему мы используем сдвиг на 1 токен между inputs и labels?
- Что означает perplexity и почему она важна для языковых моделей?
- Зачем нужен gradient clipping?
- Как learning rate scheduler помогает в обучении?


Запустим финальный прогон с наилучшими, по нашему мнению, параметрами:

In [ ]:
run_exp("Final_pretrain", task, gradient_accumulation=4, num_warmup_steps=20, scheduler="cosine_with_warmup")

Для оценки финальной модели вычисли м perplexity. Вообще наилучшая практика - оценка на одном или нескольких downstream датасетах, но это достаточно затратно.

In [ ]:
# Вычисляем финальную perplexity
final_val_loss, final_val_ppl = validate(model, val_loader, device)
print(f"Финальная валидационная perplexity: {final_val_ppl:.2f}")

# Для сравнения: perplexity предобученной модели на наших данных
model.eval()
pretrained_loss = 0
pretrained_tokens = 0

with torch.no_grad():
    for batch in val_loader:
        batch = batch.to(device)
        inputs = batch[:, :-1]
        labels = batch[:, 1:]
        
        outputs = model(inputs, labels=labels)
        loss = outputs.loss
        
        mask = (labels != tokenizer.pad_token_id).float()
        num_tokens = mask.sum().item()
        
        pretrained_loss += loss.item() * num_tokens
        pretrained_tokens += num_tokens

pretrained_avg_loss = pretrained_loss / pretrained_tokens if pretrained_tokens > 0 else 0
pretrained_ppl = calculate_perplexity(pretrained_avg_loss)

print(f"Perplexity предобученной GPT-2 на наших данных: {pretrained_ppl:.2f}")


Вопросы:
- Почему наша маленькая модель может генерировать менее качественный текст, чем предобученная?
- Как можно улучшить качество модели на downstream задачах без увеличения размера?
- Что такое perplexity и как интерпретировать её значения?


In [ ]:
# Сохранение модели
save_path = "./gpt2_pretrained"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
print(f"Модель сохранена в {save_path}")

# Загрузка модели (пример)
# loaded_model = GPT2LMHeadModel.from_pretrained(save_path)
# loaded_tokenizer = GPT2Tokenizer.from_pretrained(save_path)


### Возможные улучшения:
1. **Gradient checkpointing**: для экономии памяти
2. **Mixed Precision Training**: использование FP16 для ускорения обучения
3. **Больше данных**: использование больших корпусов текста и большего числа эпох
5. **Fine-tuning**: дообучение на специфических данных


Источники:
1) лекция Андрея Карпатого [Let's reproduce GPT-2 (124M)](https://www.youtube.com/watch?v=l8pRSuU81PU)
2) [Smol training playbook](https://huggingface.co/spaces/HuggingFaceTB/smol-training-playbook)
